In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Model trained with the small dataset
This model (model_trained.zip) has been trained only with the SQL dataset. It has a

**Character Error rate of 2.91% and Word accuracy of 95.44%. (seems overfitted)**

#Integrate SimpleHTR with word beam search
Using CTCWordBeamSearch


In [ ]:
!rm -rf CTCWordBeamSearch

In [ ]:
!git clone https://github.com/githubharald/CTCWordBeamSearch
%cd ./CTCWordBeamSearch
!pip install .
%cd /content

#Infer the model

In [2]:
!cp '/content/drive/MyDrive/TFG/TrainSimpleHTR/requirements.txt' '.'
!pip install -r requirements.txt
!pip install path

     |████████████████████████████████| 184kB 11.2MB/s 
     |████████████████████████████████| 286kB 20.2MB/s 
     |████████████████████████████████| 12.4MB 240kB/s 
     |████████████████████████████████| 49.5MB 57kB/s 
     |████████████████████████████████| 394.7MB 46kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: editdistance 0.5.3
    Uninstalling editdistance-0.5.3:
      Successfully uninstalled editdistance-0.5.3
  Found existing installation: lmdb 0.99
    Uninstalling lmdb-0.99:
      Successfully uninstalled lmdb-0.99
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow

In [3]:
!cp '/content/drive/MyDrive/TFG/HTG/output.zip' '.'
!unzip './output.zip'

!cp '/content/drive/MyDrive/TFG/TrainSimpleHTR/model_trained.zip' '.'
!unzip './model_trained.zip'
# !cp -r '/content/drive/MyDrive/TFG/TrainSimpleHTR/model' './model'
# !cp -r '/content/drive/MyDrive/TFG/TrainSimpleHTR/data' './data'
!cp -r '/content/drive/MyDrive/TFG/TrainSimpleHTR/src' './src'

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: content/output/line165/word23_img1.png  
  inflating: content/output/line165/word12_img4.png  
 extracting: content/output/line165/word30_img3.png  
  inflating: content/output/line165/word29_img0.png  
  inflating: content/output/line165/word0_img4.png  
 extracting: content/output/line165/word4_img0.png  
 extracting: content/output/line165/word18_img0.png  
 extracting: content/output/line165/word25_img1.png  
 extracting: content/output/line165/word5_img1.png  
 extracting: content/output/line165/word31_img1.png  
  inflating: content/output/line165/word27_img4.png  
 extracting: content/output/line165/word11_img2.png  
 extracting: content/output/line165/word35_img1.png  
  inflating: content/output/line165/word19_img1.png  
 extracting: content/output/line165/word1_img4.png  
 extracting: content/output/line165/word27_img1.png  
  inflating: content/output/line165/word13_img3.png  
  inflating: content/outp

In [4]:
%cd ./src
!mkdir errors

/content/src


In [5]:
import os
from openpyxl import Workbook
 
import argparse
import json
 
import cv2
import editdistance
from path import Path
 
from DataLoaderIAM import DataLoaderIAM, Batch
from Model import Model, DecoderType
from SamplePreprocessor import preprocess
 
from tqdm import tqdm
 
DECODER_TYPE = DecoderType.BeamSearch # DecoderType.BestPath    OR    DecoderType.BeamSearch    OR    DecoderType.WordBeamSearch
NUM_IMGS_PER_WORD = 5
#NUM_LINES = 445
 
class FilePaths:
    "filenames and paths to data"
    fnCharList = '../model/wordCharList.txt'
    fnSummary = '../model/summary.json'
    fnInfer = '../content/output/'    # Output folder from the HTG
    fnCorpus = '../data/corpus.txt' # corpus.txt is the original, corpusGood2.txt is the modified version
 
 
 
def infer(model, fnImg):
    "recognize text in image provided by file path"
    img = preprocess(cv2.imread(fnImg, cv2.IMREAD_GRAYSCALE), Model.imgSize)
    batch = Batch(None, [img])
    (recognized, probability) = model.inferBatch(batch, True)
    #print(f'Recognized: "{recognized[0]}"')
    #print(f'Probability: {probability[0]}')
 
    return (recognized, probability)
 
 
def main():
 
    # infer text on test image
    model = Model(open(FilePaths.fnCharList).read(), DECODER_TYPE, mustRestore=True, dump=False)
    
    wb = Workbook() # Work with .xlsx files (Excel)
    #imgsdict = {"0":"A", "1":"B", "2":"C", "3":"D", "4":"E"}
    err = 0
    # Loop on all images calling infer() to get the output
    for root, dirs, files in os.walk(FilePaths.fnInfer):
        for directory in tqdm(dirs):
            err_dir = ""
            wr = wb.create_sheet(directory)
            wp = wb.create_sheet(directory + "_prob")
            for _, _, files in os.walk(FilePaths.fnInfer + directory):
                cellsR = [[0 for i in range(NUM_IMGS_PER_WORD)] for j in range(len(files)//NUM_IMGS_PER_WORD)]
                cellsP = [[0 for i in range(NUM_IMGS_PER_WORD)] for j in range(len(files)//NUM_IMGS_PER_WORD)]
                for file in files:
                    word = file.split('_')[0][4:]
                    img  = file.split('_')[1][3:].split('.')[0]
                    try:
                        recprob = infer(model, FilePaths.fnInfer + directory + "/" + file) # Infer the image and get the recognized and probability as a tuple
                    except:
                        err += 1
                        err_dir = err_dir + file + ' '
                        # print("Error in line: ", directory, file, "Total errors: ", err)
                        raise
                    try:
                      cellsR [int(word)][int(img)] = recprob[0][0]
                      cellsP [int(word)][int(img)] = recprob[1][0]
                    except:
                      print("ERROR IN IMAGE!! ")
                      print(directory, word, img, str(len(cellsR)), str(len(cellsR[0])), recprob)
                      err_dir = err_dir + file + ' '
                      pass
                for w in range(len(cellsR)):
                    wr.append(cellsR[w])
                    wp.append(cellsP[w])
                wb.save("inferOutput.xlsx")
            if err_dir != "":
              with open("./errors/" +directory + "_errors.txt", "w") as erf:
                erf.write(err_dir)
        break
    print("FINISHED. TOTAL ERRORS: ", err)
 
 
if __name__ == '__main__':
    main()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/normalization.py:308: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  '`tf.layers.batch_normalization` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Python: 3.7.10 (default, Feb 20 2021, 21:17:23) 
[GCC 7.5.0]
Tensorflow: 2.4.0
Init with stored values from ../model/snapshot-10
INFO:tensorflow:Restoring parameters from ../model/snapshot-10


100%|██████████| 446/446 [1:33:45<00:00, 12.61s/it]

FINISHED. TOTAL ERRORS:  0


In [6]:
 
from google.colab import files
!zip -r errors.zip errors
files.download("errors.zip")
!cp 'errors.zip' '/content/drive/MyDrive/TFG/TrainSimpleHTR/SIM_BeamSearch/errors.zip'

  adding: errors/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
from google.colab import files
files.download("inferOutput.xlsx")
 
!cp 'inferOutput.xlsx' '/content/drive/MyDrive/TFG/TrainSimpleHTR/SIM_BeamSearch/inferOutput.xlsx'

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>